In [1]:
!pip3 install "tira>=0.0.139" ir-datasets "python-terrier==0.10.0"

In [2]:
!pip3 install unidecode
!pip3 install symspellpy

In [3]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

c:\Users\timko\anaconda3\envs\wir\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



### Step 2: Load the dataset

In [4]:
import pyterrier as pt

pt_dataset = pt.get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')

In [5]:
from importlib import reload
from typing import Iterable
import os
import data_cleaning
reload(data_cleaning)

class DataCleaningIter(Iterable):
    def __init__(self, dataset_iter, cleaner_function) -> None:
        self.dataset_iter = iter(dataset_iter)
        self.cleaner_function = cleaner_function
    
    def __iter__(self):
        return self
    
    def __next__(self):
        item = next(self.dataset_iter)
        item["text"] = self.cleaner_function(item["text"])
        return item

## Add BM25 Baseline

In [7]:
experiments = []
names = []
data_cleaning_iter = DataCleaningIter(pt_dataset.get_corpus_iter(), lambda x: x)
indexer = pt.IterDictIndexer(
    index_path=os.getcwd() + os.sep + f"index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)
index = indexer.index(data_cleaning_iter)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
experiments.append(bm25)
names.append("BM25")

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:  38%|███▊      | 25973/68261 [00:08<00:11, 3726.68it/s]

23:52:31.584 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6114613) - further warnings are suppressed


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents: 100%|██████████| 68261/68261 [00:19<00:00, 3424.92it/s]


23:52:44.604 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


## Add Tests

In [ ]:
from data_cleaning import replace_spelling_mistakes


data_cleaning_iter = DataCleaningIter(pt_dataset.get_corpus_iter(), replace_spelling_mistakes)
indexer = pt.IterDictIndexer(
    index_path=os.getcwd() + os.sep + f"index-spelling",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)
index = indexer.index(data_cleaning_iter)
bm25_spelling = pt.BatchRetrieve(index, wmodel="BM25")
experiments.append(bm25_spelling)
names.append("BM25+replace_spelling_mistakes")

### Step 5: Evaluate your run

In [9]:


experiment_results = pt.Experiment(
    experiments,
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    eval_metrics = ["map", "recip_rank", "ndcg_cut_10", "P_1", "P_5", "P_10"],
    names=names
)

In [10]:
experiment_results

,name,map,recip_rank,ndcg_cut_10,P_1,P_5,P_10
0,BM25,0.412718,0.786653,0.489469,0.701031,0.622680,0.574227
1,replace_spelling_mistakes,0.356378,0.721487,0.438045,0.628866,0.558763,0.522680
